# Project: Wrangling and Analyze Data

In [1]:
# Import dependencies
import requests
import os
import json
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [2]:
path_csv = 'https://raw.githubusercontent.com/lustraka/Data_Analysis_Workouts/main/Analyse_Twitter_Data/'
dfa = pd.read_csv(path_csv+'twitter-archive-enhanced.csv')
dfa.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [3]:
url_tsv = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url_tsv)
with open('image-predictions.tsv', 'wb') as file:
  file.write(r.content)
dfi = pd.read_csv('image-predictions.tsv', sep='\t')
dfi.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [9]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
from timeit import default_timer as timer

count = 0
fails_dict =  {}
start = timer()

if 'tweet_json.txt' in os.listdir():
  os.remove('tweet_json.txt')

with open('tweet_json.txt', 'a') as file:
  for tweet_id in dfa.tweet_id.values:
    count += 1
    print(str(count) + ': ' + str(tweet_id))
    try:
      status = api.get_status(tweet_id, tweet_mode='extended')._json
      print("Success")
      file.write(json.dumps(status, ensure_ascii=False)+'\n')
    except tweepy.TweepError as e:
      print('Fail')
      fails_dict[tweet_id] = e
      pass
    except e:
      print('Fail', e)
end = timer()
print(f'Elapsed time: {end - start}')
print(fails_dict)

Data gathered form Twitter API:

| Attribute | Type | Description |
| --- | :-: | --- |
| id | int | The integer representation of unique identifier for this Tweet |
| retweet_count | int | Number of times this Tweet has been retweeted. |
| favorite_count | int | *Nullable*. Indicates approximately how many times this tweet has been liked by Twitter users. |

Reference: [Tweepy docs: Tweet Object](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet)



In [11]:
df_tweets = []
with open('tweet_json.txt', 'r') as file:
  line = file.readline()
  while line:
    status = json.loads(line)
    df_tweets.append({'tweet_id': status['id'], 'retweet_count': status['retweet_count'], 'favorite_count': status['favorite_count']})
    line = file.readline()
dft = pd.DataFrame(df_tweets)
dft.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7164,34458
1,892177421306343426,5385,29870
2,891815181378084864,3551,22490
3,891689557279858688,7383,37671
4,891327558926688256,7922,35988


In [12]:
# Store dataframes for further processing
# Import dependencies
from sqlalchemy import create_engine
# Create SQLAlchemy Engine and empty database
engine = create_engine('sqlite:///weratedogsdata.db')
# Store dataframes in database
dfa.to_sql('dba', engine, index=False)
dfi.to_sql('dbi', engine, index=False)
dft.to_sql('dbt', engine, index=False)

In [ ]:
url_db = 'https://github.com/lustraka/Data_Analysis_Workouts/blob/main/Analyse_Twitter_Data/weratedogsdata.db?raw=true'
r = requests.get(url_db)
with open('weratedogsdata.db', 'wb') as file:
  file.write(r.content)

In [2]:
from sqlalchemy import create_engine
# Create SQLAlchemy Engine and empty database
engine = create_engine('sqlite:///weratedogsdata.db')

In [12]:
dfa = pd.read_sql('SELECT * FROM dba', engine)
dfa.shape

(2356, 17)

In [13]:
dfi = pd.read_sql('SELECT * FROM dbi', engine)
dfi.shape

(2075, 12)

In [14]:
dft = pd.read_sql('SELECT * FROM dbt', engine)
dft.shape

(2327, 3)

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



### Quality issues
1.

2.

3.

4.

5.

6.

7.

8.

### Tidiness issues
1.

2.

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data


### Issue #1:

#### Define:

#### Code

#### Test

### Issue #2:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization